<a href="https://colab.research.google.com/github/abdulgaffarshaik/infosys-internship/blob/master/SQLTask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# STEP 1: Import libraries
import sqlite3
import pandas as pd

# STEP 2: Create in-memory SQLite DB
conn = sqlite3.connect(":memory:")

# STEP 3: Create cyber_logs table
conn.execute("""
CREATE TABLE cyber_logs (
    id INTEGER PRIMARY KEY,
    user TEXT,
    ip_address TEXT,
    attack_type TEXT,
    country TEXT,
    src_bytes INT,
    dst_bytes INT,
    num_failed_logins INT,
    num_success_logins INT,
    port INT,
    timestamp TEXT,
    threat_text TEXT
);
""")

# STEP 4: Insert sample rows
sample_data = [
(1, 'alice', '192.168.1.10', 'DoS', 'USA', 2000, 500, 3, 1, 80, '2023-01-01 10:00:00', 'Suspicious login attempt'),
(2, 'bob', '192.168.1.11', 'Phishing', 'India', 3000, 700, 6, 0, 25, '2023-01-02 11:00:00', 'Phishing email detected'),
(3, 'eve', '192.168.1.12', 'PortScan', 'China', 1500, 200, 1, 2, 22, '2023-01-03 12:30:00', 'Multiple ports accessed'),
(4, 'alice', '192.168.1.10', 'DoS', 'USA', 2000, 500, 3, 1, 80, '2023-01-01 10:00:00', 'Duplicate log for testing'),
(5, 'charlie', '192.168.1.13', 'Malware', 'Germany', -500, 400, 2, 2, 443, '2023-01-04 09:15:00', 'Invalid negative bytes')
]

conn.executemany("""
INSERT INTO cyber_logs VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
""", sample_data)
conn.commit()

# STEP 5: Helper function to run SQL + return Pandas table
def run_query(query):
    return pd.read_sql(query, conn)


In [8]:
# 1. Remove duplicate log entries
conn.execute("""
DELETE FROM cyber_logs
WHERE id NOT IN (
    SELECT MIN(id) FROM cyber_logs GROUP BY user, ip_address, attack_type, timestamp
)
""")
run_query("SELECT * FROM cyber_logs")

,id,user,ip_address,attack_type,country,src_bytes,dst_bytes,num_failed_logins,num_success_logins,port,timestamp,threat_text
0,1,alice,192.168.1.10,DoS,USA,2000,500,3,1,80,2023-01-01 10:00:00,Suspicious login attempt
1,2,bob,192.168.1.11,Phishing,India,3000,700,6,0,25,2023-01-02 11:00:00,Phishing email detected
2,3,eve,192.168.1.12,PortScan,China,1500,200,1,2,22,2023-01-03 12:30:00,Multiple ports accessed
3,5,charlie,192.168.1.13,Malware,Germany,-500,400,2,2,443,2023-01-04 09:15:00,Invalid negative bytes


In [9]:
# 2. Replace NULL src_bytes with 1000
conn.execute("UPDATE cyber_logs SET src_bytes = 1000 WHERE src_bytes IS NULL")
run_query("SELECT * FROM cyber_logs")

,id,user,ip_address,attack_type,country,src_bytes,dst_bytes,num_failed_logins,num_success_logins,port,timestamp,threat_text
0,1,alice,192.168.1.10,DoS,USA,2000,500,3,1,80,2023-01-01 10:00:00,Suspicious login attempt
1,2,bob,192.168.1.11,Phishing,India,3000,700,6,0,25,2023-01-02 11:00:00,Phishing email detected
2,3,eve,192.168.1.12,PortScan,China,1500,200,1,2,22,2023-01-03 12:30:00,Multiple ports accessed
3,5,charlie,192.168.1.13,Malware,Germany,-500,400,2,2,443,2023-01-04 09:15:00,Invalid negative bytes


In [10]:
# 3. Normalize IP addresses (trim spaces)
conn.execute("UPDATE cyber_logs SET ip_address = TRIM(ip_address)")
run_query("SELECT DISTINCT ip_address FROM cyber_logs")


,ip_address
0,192.168.1.10
1,192.168.1.11
2,192.168.1.12
3,192.168.1.13


In [11]:
# 4. Remove corrupted rows (negative values)
conn.execute("DELETE FROM cyber_logs WHERE src_bytes < 0 OR dst_bytes < 0")
run_query("SELECT * FROM cyber_logs")


,id,user,ip_address,attack_type,country,src_bytes,dst_bytes,num_failed_logins,num_success_logins,port,timestamp,threat_text
0,1,alice,192.168.1.10,DoS,USA,2000,500,3,1,80,2023-01-01 10:00:00,Suspicious login attempt
1,2,bob,192.168.1.11,Phishing,India,3000,700,6,0,25,2023-01-02 11:00:00,Phishing email detected
2,3,eve,192.168.1.12,PortScan,China,1500,200,1,2,22,2023-01-03 12:30:00,Multiple ports accessed


In [12]:
# 5. Standardize timestamp format
run_query("SELECT id, strftime('%Y-%m-%d %H:%M:%S', timestamp) AS standardized_time FROM cyber_logs")


,id,standardized_time
0,1,2023-01-01 10:00:00
1,2,2023-01-02 11:00:00
2,3,2023-01-03 12:30:00


In [13]:
# 6. Clean text (remove '!' chars)
conn.execute("UPDATE cyber_logs SET threat_text = replace(threat_text, '!', '')")
run_query("SELECT id, threat_text FROM cyber_logs")


,id,threat_text
0,1,Suspicious login attempt
1,2,Phishing email detected
2,3,Multiple ports accessed


In [14]:
# 7. Remove outliers (failed logins > 10)
conn.execute("DELETE FROM cyber_logs WHERE num_failed_logins > 10")
run_query("SELECT * FROM cyber_logs")


,id,user,ip_address,attack_type,country,src_bytes,dst_bytes,num_failed_logins,num_success_logins,port,timestamp,threat_text
0,1,alice,192.168.1.10,DoS,USA,2000,500,3,1,80,2023-01-01 10:00:00,Suspicious login attempt
1,2,bob,192.168.1.11,Phishing,India,3000,700,6,0,25,2023-01-02 11:00:00,Phishing email detected
2,3,eve,192.168.1.12,PortScan,China,1500,200,1,2,22,2023-01-03 12:30:00,Multiple ports accessed


In [15]:
# 8. Count failed vs successful logins per user
run_query("""
SELECT user, SUM(num_failed_logins) AS failed, SUM(num_success_logins) AS success
FROM cyber_logs GROUP BY user
""")


,user,failed,success
0,alice,3,1
1,bob,6,0
2,eve,1,2


In [16]:
# 9. Top 10 attack types
run_query("""
SELECT attack_type, COUNT(*) AS frequency
FROM cyber_logs
GROUP BY attack_type
ORDER BY frequency DESC
LIMIT 10
""")


,attack_type,frequency
0,PortScan,1
1,Phishing,1
2,DoS,1


In [17]:
# 10. Avg src vs dst traffic
run_query("SELECT AVG(src_bytes) AS avg_src, AVG(dst_bytes) AS avg_dst FROM cyber_logs")


,avg_src,avg_dst
0,2166.666667,466.666667


In [18]:
# 11. Anomaly score (z-score approx)
run_query("""
SELECT user, num_failed_logins,
       (num_failed_logins - (SELECT AVG(num_failed_logins) FROM cyber_logs)) * 1.0 /
       (SELECT AVG(num_failed_logins) FROM cyber_logs) AS anomaly_score
FROM cyber_logs
""")


,user,num_failed_logins,anomaly_score
0,alice,3,-0.1
1,bob,6,0.8
2,eve,1,-0.7


In [19]:
# 12. Cluster IPs (frequency)
run_query("""
SELECT ip_address, COUNT(*) AS access_count
FROM cyber_logs GROUP BY ip_address ORDER BY access_count DESC
""")


,ip_address,access_count
0,192.168.1.12,1
1,192.168.1.11,1
2,192.168.1.10,1


In [20]:
# 13. Port scanning attempts
run_query("""
SELECT port, COUNT(*) AS attempts
FROM cyber_logs GROUP BY port ORDER BY attempts DESC
""")


,port,attempts
0,80,1
1,25,1
2,22,1


In [21]:
# 14. Attack type vs country
run_query("""
SELECT attack_type, country, COUNT(*) AS occurrences
FROM cyber_logs GROUP BY attack_type, country ORDER BY occurrences DESC
""")


,attack_type,country,occurrences
0,DoS,USA,1
1,Phishing,India,1
2,PortScan,China,1


In [22]:
# 15. Predict intrusions
run_query("""
SELECT id, user,
CASE WHEN num_failed_logins > 5 THEN 'Potential Intrusion' ELSE 'Normal' END AS prediction
FROM cyber_logs
""")


,id,user,prediction
0,1,alice,Normal
1,2,bob,Potential Intrusion
2,3,eve,Normal


In [23]:
# 16. Classify request malicious vs benign
run_query("""
SELECT id, user,
CASE WHEN src_bytes > 2500 OR dst_bytes > 500 THEN 'Malicious' ELSE 'Benign' END AS request_status
FROM cyber_logs
""")


,id,user,request_status
0,1,alice,Benign
1,2,bob,Malicious
2,3,eve,Benign


In [24]:
# 17. Forecast attack trends (by day)
run_query("""
SELECT date(timestamp) AS attack_day, COUNT(*) AS attack_count
FROM cyber_logs GROUP BY attack_day ORDER BY attack_day
""")


,attack_day,attack_count
0,2023-01-01,1
1,2023-01-02,1
2,2023-01-03,1


In [25]:
# 18. Predict phishing attempts
run_query("""
SELECT id, user,
CASE WHEN threat_text LIKE '%phish%' THEN 'Phishing Suspected' ELSE 'Normal' END AS phishing_flag
FROM cyber_logs
""")


,id,user,phishing_flag
0,1,alice,Normal
1,2,bob,Phishing Suspected
2,3,eve,Normal


In [26]:
# 19. Suspicious login attempts
run_query("""
SELECT id, user,
CASE WHEN num_failed_logins > 3 AND num_success_logins = 0 THEN 'Suspicious Login' ELSE 'Normal' END AS login_status
FROM cyber_logs
""")


,id,user,login_status
0,1,alice,Normal
1,2,bob,Suspicious Login
2,3,eve,Normal


In [27]:
# 20. Detect zero-day attacks
run_query("""
SELECT id, attack_type,
CASE WHEN attack_type IN (
    SELECT attack_type FROM cyber_logs GROUP BY attack_type HAVING COUNT(*) < 2
) THEN 'Zero-Day Suspected' ELSE 'Known Attack' END AS anomaly_flag
FROM cyber_logs
""")


,id,attack_type,anomaly_flag
0,1,DoS,Zero-Day Suspected
1,2,Phishing,Zero-Day Suspected
2,3,PortScan,Zero-Day Suspected
